<a href="https://colab.research.google.com/github/EmineKurucu/LLMProject/blob/main/LLMProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kütüphaneler eklendi**

In [16]:
!pip install openai
!pip install PyPDF2
!apt-get install libnanomsg-dev
!pip install nanomsg
!pip install oi

import openai
import os
from google.colab import files
import pandas as pd
import PyPDF2
import oi



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libnanomsg-dev is already the newest version (1.1.5+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
